<a href="https://colab.research.google.com/github/AlphaKhaw/eco-predict/blob/main/notebooks/gemini_pro_vision_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploration of `Gemini-Pro-Vision`

### Table of Contents

1. [Download and Import Packages](#import)     
2. [Authentication and Set project](#authenticate)
3. [Helper Functions](#helper)    
4. [Load and Process `MMHS150K` Dataset](#MMHS150K)
5. [Load Model - `gemini-pro-vision`](#model)  
6. [Inference and Evaluation with `MMHS150K` Dataset - Binary Classification](#binary)   

**Description**



##<b>1. Download and Import Packages<a class="anchor" id="import"></a></b>

**Download Packages:**

In [1]:
!pip install numpy pandas tqdm scikit-learn --quiet
!pip install -q -U google-generativeai --quiet

**Import Packages:**

In [2]:
import os
import PIL.Image
import subprocess
import textwrap
from google.colab import auth, drive
from google.cloud import storage
from tqdm import tqdm

import google.generativeai as genai
from google.colab import userdata

import numpy as np
import pandas as pd
from IPython.display import display
from IPython.display import Markdown
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score
)


drive.mount('/content/drive')

Mounted at /content/drive


##<b>2. Authentication and Set Project<a class="anchor" id="authenticate"></a></b>

In [3]:
auth.authenticate_user()

**Set your Google Cloud project ID:**


In [4]:
project_id = 'impressive-bay-414615'
!gcloud config set project {project_id}

Updated property [core/project].


In [5]:
!gsutil ls gs://mmhs150k

gs://mmhs150k/MMHS150K_GT.json
gs://mmhs150k/MMHS150K_readme.txt
gs://mmhs150k/hatespeech_keywords.txt
gs://mmhs150k/img_resized/
gs://mmhs150k/img_txt/
gs://mmhs150k/processed/
gs://mmhs150k/splits/


**Load in data from GCP Bucket:**

In [6]:
# Define the folder and file paths
img_resized_folderpath = "/content/drive/MyDrive/img_resized"
processed_folderpath = "/content/drive/MyDrive/processed"
test_json_filepath = "/content/drive/MyDrive/processed/test.json"

# Check if the folders already exist, create them if they don't
if not os.path.exists(img_resized_folderpath):
    os.makedirs(img_resized_folderpath)

    # Perform the gsutil command to copy the img_resized
    subprocess.run("gsutil -m cp -r gs://mmhs150k/img_resized/img_resized /content/drive/MyDrive/", shell=True)

if not os.path.exists(processed_folderpath):
    os.makedirs(processed_folderpath)

# Check if the test.json file already exists, copy it if it doesn't
if not os.path.isfile(test_json_filepath):

    # Perform the gsutil command to copy the test.json file
    subprocess.run("gsutil cp gs://mmhs150k/processed/test.json /content/drive/MyDrive/processed/test.json", shell=True)

**Setup API key:**

In [7]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

##<b>3. Helper Functions <a class="anchor" id="helper"></a></b>


Helper functions for data processing and evalution.

In [8]:
def evaluate(
    true_labels: list, y_pred: np.ndarray, y_proba: np.ndarray = None
) -> None:
    """
    Evaluates and logs the performance of a classifier using multiple metrics.
    This function calculates and logs the average accuracy, balanced accuracy,
    and F1 score for predictions made by a classifier.

    Args:
        true_labels (list): A list of true label sets. Each set in the list
                            should correspond to a set of true class labels.
        y_pred (np.ndarray): An array of predicted labels. These predictions are
                             compared against each set of true labels.
        y_proba (np.ndarray): An array of predicted probabilities. These
                              predictions are compared against each set of true
                              labels.
    """
    accuracy_scores, auroc_scores, balanced_accuracy_scores, f1_scores = (
        [], [], [], []
    )
    for true_label in true_labels:
        accuracy_scores.append(accuracy_score(true_label, y_pred))
        balanced_accuracy_scores.append(
            balanced_accuracy_score(true_label, y_pred)
        )
        f1_scores.append(f1_score(true_label, y_pred, average="macro"))

        if y_proba is not None:
            auroc_scores.append(roc_auc_score(true_label, y_proba))

    if y_proba is not None:
        print(
            f"AUROC Score: {round(np.mean(auroc_scores)*100, 2)}%"
        )

    print(
        f"Average Accuracy Score: {round(np.mean(accuracy_scores)*100, 2)}%"
    )
    print(
        f"Balanced Accuracy Score: {round(np.mean(accuracy_scores)*100, 2)}%"
    )
    print(f"Average F1 Score: {round(np.mean(f1_scores)*100, 2)}%")


def read_image(filepath: str) -> PIL.Image.Image:
    """
    Reads an image from a specified file path using PIL (Python Imaging Library)
    and returns the image object.

    This function opens an image file and loads it into a PIL Image object,
    which can then be used for image processing or display within Python.

    Args:
        filepath (str): The path to the image file that is to be read. This can
                        be a path to a local file or a URL.

    Returns:
        PIL.Image.Image: An image object containing the image data loaded from
                         the specified file.

    Raises:
        FileNotFoundError: If the file specified by `filepath` does not exist.
        PIL.UnidentifiedImageError: If the file cannot be opened and identified
                                    as a supported image format.

    Example:
        >>> image = read_image('/path/to/image.jpg')
        >>> image.show()  # Displays the image using the default image viewer.

    Note:
        Supported image formats depend on the version of PIL or Pillow
        (a fork of PIL) being used. Common formats include JPEG, PNG, BMP, and
        GIF.
    """
    return PIL.Image.open(filepath)

##<b>4. Load and Process `MMHS150K` Dataset<a class="anchor" id="MMHS150K"></a></b>

**Read in Test Dataset:**

In [9]:
dataframe = pd.read_json(
    "/content/drive/MyDrive/processed/test.json", lines=True
)
dataframe['image'] = dataframe['image'].apply(
    lambda x: x.replace("../data/MMHS150K", "img_resized")
)
dataframe

,image,text,label_0,label_1,label_2,label_0_str,label_1_str,label_2_str,binary_label_0,binary_label_1,binary_label_2,binary_label_0_str,binary_label_1_str,binary_label_2_str
0,img_resized/img_resized/1113114611979636737.jpg,They' ll pay for this,2,2,2,Sexist,Sexist,Sexist,1,1,1,Hate,Hate,Hate
1,img_resized/img_resized/1043320121610174465.jpg,"""SHERMAN DUPON HELMSLEY [SDH""",1,1,1,Racist,Racist,Racist,1,1,1,Hate,Hate,Hate
2,img_resized/img_resized/1062965467680108544.jpg,OK? JAPANHDV COM,0,0,0,NotHate,NotHate,NotHate,0,0,0,NotHate,NotHate,NotHate
3,img_resized/img_resized/1057692490906722310.jpg,33thb,0,0,0,NotHate,NotHate,NotHate,0,0,0,NotHate,NotHate,NotHate
4,img_resized/img_resized/1055137831570628608.jpg,10 2y '98,0,0,0,NotHate,NotHate,NotHate,0,0,0,NotHate,NotHate,NotHate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213,img_resized/img_resized/1110340935274192902.jpg,RBERR RBERA RBERR MADE WITH SPLIT PIC,0,0,0,NotHate,NotHate,NotHate,0,0,0,NotHate,NotHate,NotHate
1214,img_resized/img_resized/1108232716951175168.jpg,YEAH I'M A PSXCHO,1,1,1,Racist,Racist,Racist,1,1,1,Hate,Hate,Hate
1215,img_resized/img_resized/1114678367695986688.jpg,4:23 .111 LTE Dying To Live Kodak Black V Like...,2,2,2,Sexist,Sexist,Sexist,1,1,1,Hate,Hate,Hate
1216,img_resized/img_resized/1113965844588105728.jpg,S/O MY LAST BITCH IM SHITTIN ON HER NIGGA NO,0,0,0,NotHate,NotHate,NotHate,0,0,0,NotHate,NotHate,NotHate


**Modify `image` column filepaths to point to Google Drive filepaths:**

In [10]:
dataframe['image'] = dataframe['image'].apply(
    lambda x: x.replace(
        "img_resized/img_resized", "/content/drive/MyDrive/img_resized"
    )
)
dataframe

,image,text,label_0,label_1,label_2,label_0_str,label_1_str,label_2_str,binary_label_0,binary_label_1,binary_label_2,binary_label_0_str,binary_label_1_str,binary_label_2_str
0,/content/drive/MyDrive/img_resized/11131146119...,They' ll pay for this,2,2,2,Sexist,Sexist,Sexist,1,1,1,Hate,Hate,Hate
1,/content/drive/MyDrive/img_resized/10433201216...,"""SHERMAN DUPON HELMSLEY [SDH""",1,1,1,Racist,Racist,Racist,1,1,1,Hate,Hate,Hate
2,/content/drive/MyDrive/img_resized/10629654676...,OK? JAPANHDV COM,0,0,0,NotHate,NotHate,NotHate,0,0,0,NotHate,NotHate,NotHate
3,/content/drive/MyDrive/img_resized/10576924909...,33thb,0,0,0,NotHate,NotHate,NotHate,0,0,0,NotHate,NotHate,NotHate
4,/content/drive/MyDrive/img_resized/10551378315...,10 2y '98,0,0,0,NotHate,NotHate,NotHate,0,0,0,NotHate,NotHate,NotHate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213,/content/drive/MyDrive/img_resized/11103409352...,RBERR RBERA RBERR MADE WITH SPLIT PIC,0,0,0,NotHate,NotHate,NotHate,0,0,0,NotHate,NotHate,NotHate
1214,/content/drive/MyDrive/img_resized/11082327169...,YEAH I'M A PSXCHO,1,1,1,Racist,Racist,Racist,1,1,1,Hate,Hate,Hate
1215,/content/drive/MyDrive/img_resized/11146783676...,4:23 .111 LTE Dying To Live Kodak Black V Like...,2,2,2,Sexist,Sexist,Sexist,1,1,1,Hate,Hate,Hate
1216,/content/drive/MyDrive/img_resized/11139658445...,S/O MY LAST BITCH IM SHITTIN ON HER NIGGA NO,0,0,0,NotHate,NotHate,NotHate,0,0,0,NotHate,NotHate,NotHate


##<b>5. Load Model - `gemini-pro-vision`<a class="anchor" id="gemini"></a></b>

In [11]:
model = genai.GenerativeModel('gemini-pro-vision')

##<b>6. Inference and Evaluation with `MMHS150K` Dataset - Binary Classification<a class="anchor" id="binary"></a></b>

**Inference:**

In [16]:
from tqdm import tqdm

# Instantiate list to store results
results = []

# Iterate over the DataFrame rows
for index, row in tqdm(dataframe.iterrows(), total=dataframe.shape[0]):

    # Construct the messages for each row
    attempt = 0
    max_attempts = 3  # Set a maximum number of retry attempts

    while attempt < max_attempts:
        try:
            response = model.generate_content(
                [
                    f"Accompanying text: {row['text']}. Given the following meme from social media, which includes both an image and accompanying text, classify the content as hateful or non-hateful. Consider both the visual and textual elements in your assessment. Strictly provide a binary classification: 0 for non-hateful and 1 for hateful.",
                    read_image(row['image'])
                ]
            )

            # If response is successful, append to results and break the loop
            results.append(response.text)
            break

        except ValueError as e:
            attempt += 1  # Increment the attempt counter
            print(f"Attempt {attempt} failed. Retrying...")

            # Append None or a placeholder to indicate failure
            if attempt == max_attempts:
                print(
                    f"Maximum attempts reached for index {index}. Skipping..."
                )
                results.append(None)


  0%|          | 2/1218 [00:09<1:35:31,  4.71s/it]

Attempt 1 failed. Retrying...
Attempt 2 failed. Retrying...


  0%|          | 3/1218 [00:13<1:24:42,  4.18s/it]

Attempt 3 failed. Retrying...
Maximum attempts reached for index 2. Skipping...


  1%|          | 9/1218 [00:38<1:26:48,  4.31s/it]

...

100%|█████████▉| 1216/1218 [1:55:20<00:09,  4.54s/it]

Attempt 1 failed. Retrying...
Attempt 2 failed. Retrying...


100%|█████████▉| 1217/1218 [1:55:33<00:06,  6.90s/it]

Attempt 3 failed. Retrying...
Maximum attempts reached for index 1216. Skipping...


100%|██████████| 1218/1218 [1:55:37<00:00,  5.70s/it]


**Evaluation:**

In [25]:
extracted_contents = []
for result in results:
    if result is not None:
        content = result.strip()
    else:
        extracted_contents.append(None)
        continue
    # Check for numeric content and convert to int
    if content.isdigit():
        extracted_contents.append(int(content))
    # Check for textual content and convert to corresponding int
    elif content.lower() == "hateful":
        extracted_contents.append(1)  # Assuming "Hateful" corresponds to 1
    elif content.lower() == "non-hateful":
        extracted_contents.append(0)  # Assuming "Non-hateful" corresponds to 0

In [28]:
true_label_0 = dataframe['binary_label_0'].apply(int).explode().tolist()
true_label_1 = dataframe['binary_label_1'].apply(int).explode().tolist()
true_label_2 = dataframe['binary_label_2'].apply(int).explode().tolist()

filtered_label_0 = [
    label for label, result in zip(true_label_0, extracted_contents)
    if result is not None
]
filtered_label_1 = [
    label for label, result in zip(true_label_1, extracted_contents)
    if result is not None
]
filtered_label_2 = [
    label for label, result in zip(true_label_2, extracted_contents)
    if result is not None
]
extracted_contents_filtered = [
    result for result in extracted_contents if result is not None
]

evaluate(
    true_labels=[filtered_label_0, filtered_label_1, filtered_label_2],
    y_pred=extracted_contents_filtered,
    y_proba=extracted_contents_filtered,
)

AUROC Score: 50.0%
Average Accuracy Score: 52.21%
Balanced Accuracy Score: 52.21%
Average F1 Score: 34.3%
